In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Load Dataset**

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Group_Chat.csv")


In [ ]:
df.dropna(inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming your dataset has columns 'Conversations' and 'story'
X = df['Conversations']
y = df['story']

# Perform train-test split with 80% training data and 20% testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 0.25 x 0.8 = 0.2 = 20% of original data

# Create train, test, and validation DataFrames
train_df = pd.DataFrame({'Conversations': X_train, 'story': y_train})
test_df = pd.DataFrame({'Conversations': X_test, 'story': y_test})
val_df = pd.DataFrame({'Conversations': X_val, 'story': y_val})


In [ ]:
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


In [ ]:
print(test_df.shape,train_df.shape,val_df.shape)

(20, 2) (64, 2) (16, 2)


In [ ]:
from datasets import DatasetDict, Dataset

# Convert train_df and val_df to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict containing the train and validation datasets
dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset,"test":test_dataset})


In [ ]:
type(dataset_dict)

datasets.dataset_dict.DatasetDict

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Conversations', 'story'],
        num_rows: 64
    })
    validation: Dataset({
        features: ['Conversations', 'story'],
        num_rows: 16
    })
    test: Dataset({
        features: ['Conversations', 'story'],
        num_rows: 20
    })
})

In [ ]:
train_df


,Conversations,story
0,"Priya: Hey everyone, how's it going?\nRahul: H...","In the digital space of their WhatsApp group,..."
1,"Aisha: Hey everyone, how's everyone doing?\nKa...",Within the confines of their digital WhatsApp ...
2,"Avni: Hey everyone, how's it going?\nKabir: Hi...","In the digital realm of their WhatsApp group,..."
3,"Aditya: Hey everyone, how's it going?\nNeha: H...","In the digital space of their WhatsApp group,..."
4,"Riya: Hey everyone, how's it going?\nAryan: Hi...",Within the digital realm of their WhatsApp gro...
...,...,...
59,"Arjun: Hey everyone, how's it going?\nMaya: Hi...","In the virtual space of their WhatsApp group,..."
60,"Tara: Hey everyone, how's it going?\nArjun: Hi...","In the digital space of their WhatsApp group, ..."
61,"Aarav: Hey everyone, how's it going?\nDia: Hi ...",Within the digital realm of their WhatsApp gro...
62,"Priya: Hey everyone, how's everyone doing toda...",Within the digital realm of their WhatsApp gr...


In [ ]:
c=df['Conversations'][0]

In [ ]:
c

"Emma: Good morning, everyone!\nLiam: Morning, Emma. How's everyone doing today?\nOlivia: I'm feeling great, thanks for asking. How about you?\nLiam: Can't complain. Just another day at the grind.\nNoah: Same here. Work, work, work!\nEmma: Well, at least we have each other to keep company.\nLiam: That's true. It's always nice to chat with you all.\nOlivia: Definitely. You guys make the workday bearable.\nNoah: Agreed. So, any exciting plans for the weekend?\nEmma: I'm thinking of going to the beach if the weather holds up.\nLiam: Sounds awesome. Mind if I tag along?\nOlivia: Count me in too! I could use some sun.\nNoah: Sorry, guys, I'm booked this weekend. Maybe next time.\nEmma: No worries, Noah. We'll catch up another time.\nLiam: Yeah, for sure. Have a great weekend, everyone!\nOlivia: You too, Liam. Enjoy the beach!\nNoah: Take care, Liam. Don't forget your sunscreen!\nEmma: Haha, noted!\nLiam: Bye, guys!\nOlivia: Bye Liam!"

In [ ]:
pipe=pipeline('summarization',model=model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
out=pipe(c)

In [ ]:
out

[{'summary_text': 'Emma, Liam and Noah all have their work cut out for them as they try to keep up with the Joneses .<n>Emma suggests a trip to the beach; Liam suggests a day at the beach .<n>Noah suggests a day at the beach; Liam suggests a day at the beach .<n>Olivia suggests a day at the beach; Liam suggests a day at the beach .'}]

In [ ]:
print(out[0]['summary_text'].replace(" .<n>", ".\n"))

Emma, Liam and Noah all have their work cut out for them as they try to keep up with the Joneses.
Emma suggests a trip to the beach; Liam suggests a day at the beach.
Noah suggests a day at the beach; Liam suggests a day at the beach.
Olivia suggests a day at the beach; Liam suggests a day at the beach .


In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def calculate_metric_on_test_ds(article_list, story_list, metric, model, tokenizer,
                               batch_size=1, device=device):
    for article, story in tqdm(zip(article_list, story_list), total=len(article_list)):
        inputs = tokenizer(article, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs.input_ids.to(device),
                                   attention_mask=inputs.attention_mask.to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces=True)
                             for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=[story])

    score = metric.compute()
    return score

In [ ]:
metric = load_metric("rouge")
score = calculate_metric_on_test_ds(
    dataset_dict['test']['Conversations'], dataset_dict['test']['story'],
    metric,model_pegasus, tokenizer, batch_size=8
)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 20/20 [00:46<00:00,  2.34s/it]


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

s=pd.DataFrame(rouge_dict, index = ['pegasus'])

In [ ]:
s

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.025631,0.0,0.025672,0.025679


In [ ]:
from nltk.tokenize import word_tokenize  # Assuming you're using NLTK for tokenization
from collections import Counter

all_conversations = ' '.join(df['Conversations'])
#Tokenize the concatenated string into words or tokens
tokens = word_tokenize(all_conversations)

# Count occurrences of each unique tokenizer
tokenizer_counts = Counter(tokens)

print("Tokenizer Counts:", tokenizer_counts)

total_tokenizers = sum(tokenizer_counts.values())
print("Total Tokenizers:", total_tokenizers)


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Conversations'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['story'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

#dataset_df_pt = df.apply(convert_examples_to_features, axis=1)
dataset_samsum_pt = dataset_dict.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
!pip install transformers[torch]
!pip install accelerate>=0.21.0

#after running this cell,restart the session for first time only

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import TrainingArguments,Trainer
"""
trainer_args = TrainingArguments(
    output_dir='pegasus1',  # Directory where checkpoints and logs will be saved
    num_train_epochs=1,  # Number of training epochs (adjust as needed)
    per_device_train_batch_size=1,  # Batch size per GPU/CPU for training
    per_device_eval_batch_size=1,  # Batch size per GPU/CPU for evaluation
    logging_steps=1,  # Log training metrics after every `logging_steps` updates
    evaluation_strategy='epoch',  # Evaluate model after each epoch
    save_steps=1e6,  # Save a checkpoint after `save_steps` updates (no intermediate checkpoints)
    gradient_accumulation_steps=1,  # Number of gradient accumulation steps (1 means no gradient accumulation)
    # Add more parameters as needed based on your training requirements
)"""
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=100, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=400, training_loss=1.1244434528052807, metrics={'train_runtime': 1928.7877, 'train_samples_per_second': 3.318, 'train_steps_per_second': 0.207, 'total_flos': 5368929629798400.0, 'train_loss': 1.1244434528052807, 'epoch': 100.0})

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_dict['test']['Conversations'], dataset_dict['test']['story'],
    metric, trainer.model, tokenizer, batch_size=2
)

100%|██████████| 20/20 [01:11<00:00,  3.58s/it]


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_dict = {rn: score[rn].mid.fmeasure for rn in rouge_names}

pd.DataFrame(rouge_dict, index=['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.013131,0.0,0.013145,0.013126


In [ ]:
## Save model
model_pegasus.save_pretrained("/content/drive/MyDrive/pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [ ]:
## Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/tokenizer")

('/content/drive/MyDrive/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/tokenizer/spiece.model',
 '/content/drive/MyDrive/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/tokenizer/tokenizer.json')

**Test the Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [ ]:
sample_text = dataset_dict["test"][5]["Conversations"]

reference = dataset_dict["test"][5]["story"]

In [ ]:
sample_text

"Riya: Hey everyone, how's it going?\nAarav: Hi Riya! Just came back from a sustainable farming workshop.\nNeha: Hi Riya and Aarav! I'm planning a neighborhood tree-planting initiative.\nRohan: Hey guys! Just finished a birdwatching expedition in the forest.\nRiya: Sustainable farming, tree-planting, and birdwatching, sounds like a day in harmony with nature!\nAarav: Sustainable farming workshops help us grow food while caring for the environment.\nNeha: Tree-planting initiatives are essential for improving air quality and promoting biodiversity.\nRohan: Birdwatching allows us to appreciate the beauty of nature and learn about local ecosystems.\nRiya: So, any plans for the weekend?\nAarav: I'm thinking of organizing a farmers' market to promote sustainable produce from the workshop.\nNeha: Count me in, Aarav! I can gather volunteers for the tree-planting event and spread awareness.\nRohan: Sounds like a plan! I'll invite some friends to join us for a birdwatching tour in the nearby san

In [ ]:
reference

" Amidst the digital connections of their virtual space, Riya sparked a lively conversation among her friends. Aarav, passionate about sustainable farming, and Neha, dedicated to environmental conservation, shared their recent activities. Rohan, finding solace in birdwatching, added his perspective to the chat. Sensing an opportunity for a weekend aligned with their shared values, Riya proposed a farmers' market, tree-planting initiative, and birdwatching tour. Excited by the idea, Aarav, Neha, and Rohan eagerly agreed. With plans set, they anticipated a weekend filled with sustainability, community building, and immersion in nature."

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

In [ ]:
print("Conversations:")
print(sample_text)


print("\nReference story:")
print(reference)


print("\nModel story:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Conversations:
Riya: Hey everyone, how's it going?
Aarav: Hi Riya! Just came back from a sustainable farming workshop.
Neha: Hi Riya and Aarav! I'm planning a neighborhood tree-planting initiative.
Rohan: Hey guys! Just finished a birdwatching expedition in the forest.
Riya: Sustainable farming, tree-planting, and birdwatching, sounds like a day in harmony with nature!
Aarav: Sustainable farming workshops help us grow food while caring for the environment.
Neha: Tree-planting initiatives are essential for improving air quality and promoting biodiversity.
Rohan: Birdwatching allows us to appreciate the beauty of nature and learn about local ecosystems.
Riya: So, any plans for the weekend?
Aarav: I'm thinking of organizing a farmers' market to promote sustainable produce from the workshop.
Neha: Count me in, Aarav! I can gather volunteers for the tree-planting event and spread awareness.
Rohan: Sounds like a plan! I'll invite some friends to join us for a birdwatching tour in the nearby 